In [17]:
!pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=72396221719006e5ea2e05cc67bb0a99969b0bc28e41ad30a055d878c5b89b7b
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [3]:
!pip install --quiet qiskit ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 12.0 MB/s eta 0:00:00


In [4]:
!pip3 install --quiet qiskit-aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 46.7 MB/s eta 0:00:00


In [5]:
from qiskit import QuantumCircuit, assemble,transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import numpy as np

In [27]:
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit.visualization import plot_circuit_layout
from qiskit import QuantumCircuit
# DNA Mapping Table
dna_mapping = {
    '00': 'A',
    '01': 'C',
    '10': 'G',
    '11': 'T'
}

# Reverse DNA Mapping Table
reverse_dna_mapping = {v: k for k, v in dna_mapping.items()}

# Rules Table
rules_table = {
    '↑': 'TGCA',  # Example rule for up arrow
    '↓': 'ACGT',  # Example rule for down arrow
    '→': 'GACT',  # Example rule for right arrow
    '←': 'CTGA'   # Example rule for left arrow
}

# Function to map bits to nucleotides
def bits_to_nucleotides(bits):
    return ''.join(dna_mapping[bits[i:i+2]] for i in range(0, len(bits), 2))

# Function to map nucleotides to bits
def nucleotides_to_bits(nucleotides):
    return ''.join(reverse_dna_mapping[nuc] for nuc in nucleotides)

# Function to divide information into blocks
def divide_into_blocks(info, block_size):
    return [info[i:i+block_size] for i in range(0, len(info), block_size)]

# Function to permute using a rule
def permute_block(block, rule):
    if not all(nuc in 'ACGT' for nuc in block):
        raise ValueError(f"Block contains invalid nucleotides: {block}")
    return ''.join(rule['ACGT'.index(nuc)] for nuc in block)

# Function to invert permutation
def invert_permutation(block, rule):
    inv_rule = ''.join(sorted(rule, key=rule.index))
    return ''.join(inv_rule['ACGT'.index(nuc)] for nuc in block)

# Function to create a quantum circuit for permutation
def create_permutation_circuit(block, rule):
    n = len(block)
    qc = QuantumCircuit(n, n)
    for i, nuc in enumerate(block):
        if nuc == 'C':
            qc.x(i)
        elif nuc == 'G':
            qc.y(i)
        elif nuc == 'T':
            qc.z(i)
    qc.barrier()
    return qc

# Function to add measurement to a circuit
def add_measurement(qc):
    n = qc.num_qubits
    qc.measure(range(n), range(n))
    return qc

# Function to simulate quantum circuit and return results
def simulate_circuit(qc):
    backend = Aer.get_backend('qasm_simulator')
    transpiled_qc = transpile(qc, backend)
    #qobj = assemble(transpiled_qc)
    result = backend.run(transpiled_qc).result()
    counts = result.get_counts()
    return counts

# Sender side
def sender(info_bits, basis):
    # Step 1: Map bits to nucleotides
    nucleotides = bits_to_nucleotides(info_bits)
    print(f"Nucleotides: {nucleotides}")

    # Divide nucleotides into blocks
    k = 4  # Example block size
    blocks = divide_into_blocks(nucleotides, k)
    print(f"Blocks: {blocks}")

    # Step 2: Encrypt blocks using permutation
    rule = rules_table[basis]
    encrypted_blocks = []
    encrypted_message_counts = {}
    for block in blocks:
        try:
            encrypted_block = permute_block(block, rule)
            encrypted_blocks.append(encrypted_block)

            # Create quantum circuit for the permutation
            qc = create_permutation_circuit(block, rule)

            # Add measurement to the circuit
            qc = add_measurement(qc)

            # Simulate the encryption circuit
            counts = simulate_circuit(qc)
            encrypted_message_counts[block] = counts

            # Print the quantum circuit
            print(f"Quantum Circuit for block {block}:")
            print(qc.draw(output='text'))

            # Plot the measurement results
            plot_histogram(counts)
            plt.show()

        except ValueError as e:
            print(f"Error permuting block: {e}")
            raise

    print(f"Encrypted blocks: {encrypted_blocks}")

    return encrypted_message_counts, basis

# Receiver side
def receiver(encrypted_message_counts, basis):
    # Step 4: Decrypt using inverse permutation
    rule = rules_table[basis]

    # Invert permutation
    inv_rule = ''.join(sorted(rule, key=rule.index))
    decrypted_blocks = []
    for encrypted_block in encrypted_message_counts:
        decrypted_block = invert_permutation(encrypted_block, inv_rule)
        decrypted_blocks.append(decrypted_block)

    decrypted_message = ''.join(decrypted_blocks)

    return decrypted_message

# Example usage
info_bits = '0001101100111010'  # Example bit string
basis = '↓'
print(f'Information bits: {info_bits}')
print(f'Basis: {basis}')
encrypted_message_counts, basis = sender(info_bits, basis)
print(f'Encrypted message counts: {encrypted_message_counts}')

decrypted_message = receiver(encrypted_message_counts, basis)
print(f'Decrypted message: {decrypted_message}')



Information bits: 0001101100111010
Basis: ↓
Nucleotides: ACGTATGG
Blocks: ['ACGT', 'ATGG']
Quantum Circuit for block ACGT:
           ░ ┌─┐         
q_0: ──────░─┤M├─────────
     ┌───┐ ░ └╥┘┌─┐      
q_1: ┤ X ├─░──╫─┤M├──────
     ├───┤ ░  ║ └╥┘┌─┐   
q_2: ┤ Y ├─░──╫──╫─┤M├───
     ├───┤ ░  ║  ║ └╥┘┌─┐
q_3: ┤ Z ├─░──╫──╫──╫─┤M├
     └───┘ ░  ║  ║  ║ └╥┘
c: 4/═════════╩══╩══╩══╩═
              0  1  2  3 
Quantum Circuit for block ATGG:
           ░ ┌─┐         
q_0: ──────░─┤M├─────────
     ┌───┐ ░ └╥┘┌─┐      
q_1: ┤ Z ├─░──╫─┤M├──────
     ├───┤ ░  ║ └╥┘┌─┐   
q_2: ┤ Y ├─░──╫──╫─┤M├───
     ├───┤ ░  ║  ║ └╥┘┌─┐
q_3: ┤ Y ├─░──╫──╫──╫─┤M├
     └───┘ ░  ║  ║  ║ └╥┘
c: 4/═════════╩══╩══╩══╩═
              0  1  2  3 
Encrypted blocks: ['ACGT', 'ATGG']
Encrypted message counts: {'ACGT': {'0110': 1024}, 'ATGG': {'1100': 1024}}


Decrypted message: ACGTATGG
Decrypted bits: 0001101100111010


